In [2]:
import ticktack                 # For the compilation of the model
import ode                      # As a method of solving the odes
import numpy as np              # For the basic functions that need to be written
from jax import numpy as jnp    # annoying shit that I wish I didn't hav to deal with 

In [3]:
parameters = np.zeros(6)            # Empty array to hold the parameters of the prodcution function 
parameters[0] = 7.044873503263437   # Mean of the sinusoidal production 
parameters[1] = 0.18                # Amplitude of the sinusoidal production 
parameters[2] = 11.0                # Period of the sinusoidal production 
parameters[3] = 1.25                # Phase of the sinusoidal production 
parameters[4] = 120.05769867244142  # Height of the super-gaussian 
parameters[5] = 12.0                # Width of the super-gaussian 

In [4]:
projection = np.zeros(11)   # An array to hold the projection of the production function 
projection[0] = 0.7         # Proportion projection into the stratosphere       
projection[1] = 0.3         # Proportion projection into the troposphere

In [5]:
def production(t):    
    """
    The production function defined as the sum of sinusoidal and super-gaussian terms 
    """
    return parameters[0] * (1 + parameters[1] * \
        jnp.sin(2 * jnp.pi / parameters[2] * t + parameters[3])) + \
        parameters[4] * jnp.exp(- (parameters[5] * (t - 775)) ** 16)

In [6]:
cbm = ticktack.load_presaved_model("Guttler14", production_rate_units="atoms/cm^2/s")
cbm.compile()   # Constructing the transfer operator 

The following array represents the position of the system following the burn in period. Instead of running this in python I just pirated the julia results because it was easier to do this.

In [16]:
u0 = np.array([135.76261605786132, 709.7591911307035,
    1191.489526709938, 3.97158546945107, 45158.65854589925,
    155.54703228960028, 634.290736403387, 423.5954241095565,
    1808.3343542055652, 665.933052175064, 7348.751626918585])

In [15]:
def dydx(y, t):
    """
    The gradient of the carbon box model.
    """
    return cbm._matrix @ y + production(t) * projection

In [17]:
ode.odeint(dydx, u0, np.arange(760.0, 790.0, 1))    # Running the ode

(DeviceArray([[1.35762616e+02, 7.09759191e+02, 1.19148953e+03,
               3.97158547e+00, 4.51586585e+04, 1.55547032e+02,
               6.34290736e+02, 4.23595424e+02, 1.80833435e+03,
               6.65933052e+02, 7.34875163e+03],
              [1.36031060e+02, 7.10354201e+02, 1.19156922e+03,
               3.97183511e+00, 4.51586497e+04, 1.55673012e+02,
               6.34290530e+02, 4.23612412e+02, 1.80833309e+03,
               6.65932631e+02, 7.34875154e+03],
              [1.35936437e+02, 7.10705470e+02, 1.19169538e+03,
               3.97224790e+00, 4.51586459e+04, 1.55783460e+02,
               6.34306623e+02, 4.23667108e+02, 1.80833219e+03,
               6.65932333e+02, 7.34875148e+03],
              [1.35508776e+02, 7.10701486e+02, 1.19182795e+03,
               3.97269286e+00, 4.51586483e+04, 1.55843314e+02,
               6.34333908e+02, 4.23742146e+02, 1.80833196e+03,
               6.65932259e+02, 7.34875147e+03],
              [1.34883867e+02, 7.10343506e+02, 1.191